# 🛠️ Mastering Tools in LangGraph: From Functions to MCP

In this notebook, we’ll build an AI agent that doesn't just talk—it **acts**. We will cover:
1. **Custom Python Tools** using the `@tool` decorator.
2. **The Prebuilt `ToolNode`** for automated execution.
3. **Model Context Protocol (MCP)** for standardized external integration.
4. **The Feedback Loop**: How agents learn from tool results.

In [ ]:
# Install necessary dependencies
!pip install -U langgraph langchain-openai langchain-mcp-adapters

## 1. Defining Real-World Python Tools
Let's create a tool for a **Travel Agent**. This agent needs to fetch weather and calculate flight carbon offsets.

In [ ]:
import os
from langchain_core.tools import tool

@tool
def get_weather(city: str):
    """Consult this tool to get the current weather for a specific city."""
    if "london" in city.lower():
        return "It's currently 15°C and rainy."
    return "It's a beautiful 25°C and sunny."

@tool
def carbon_offset_calculator(miles: int):
    """Calculates the carbon offset cost in USD for a flight based on mileage."""
    # Simple logic: $0.05 per mile
    return f"The carbon offset cost for {miles} miles is ${miles * 0.05:.2f}."

tools = [get_weather, carbon_offset_calculator]

## 2. Setting up the Prebuilt ToolNode
LangGraph's `ToolNode` acts as the "hands" of our agent. It listens for `tool_calls` in the LLM's message and runs them automatically.

In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)

## 3. Integrating MCP (Model Context Protocol)
MCP allows you to connect to a universe of servers (SQLite, Slack, GitHub) without writing custom wrappers for every API.

*Note: In a real scenario, you'd have an MCP server running. Here is how you'd connect to it.*

In [ ]:
from langchain_mcp_adapters.tools import load_mcp_tools

try:
    # Example: Connecting to a local MCP server managing a database
    # mcp_tools = load_mcp_tools("http://localhost:8000/mcp")
    # all_tools = tools + mcp_tools
    all_tools = tools # Staying with local tools for this demo
    print("MCP tools loaded successfully!")
except Exception as e:
    all_tools = tools
    print(f"MCP not found, proceeding with basic tools. Error: {e}")

## 4. Building the Graph
We use `MessagesState` to keep track of the conversation history, including the tool outputs.

In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import tools_condition
from langchain_openai import ChatOpenAI

# Bind the tools to the model so it knows they exist
llm = ChatOpenAI(model="gpt-4o", api_key="YOUR_OPENAI_API_KEY")
llm_with_tools = llm.bind_tools(all_tools)

def chatbot(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("agent", chatbot)
builder.add_node("tools", tool_node)

# Define edges
builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", tools_condition)
builder.add_edge("tools", "agent") # Circle back to agent to summarize results

graph = builder.compile()

## 5. Running the Agent
Watch how the agent calls multiple tools to answer a complex request.

In [ ]:
inputs = {"messages": [("user", "I'm flying 2000 miles to London. What's the weather there and how much will my carbon offset be?")]}

for output in graph.stream(inputs, stream_mode="updates"):
    for node, values in output.items():
        print(f"--- Node: {node} ---")
        last_msg = values["messages"][-1]
        if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
            print(f"Action: Calling tools { [t['name'] for t in last_msg.tool_calls] }")
        else:
            print(f"Content: {last_msg.content[:100]}...")